# 3.3 signac-flow aggregation example

## About

This notebook contains a minimal example for running a signac-flow's **aggregation** feature which is not released yet.
The example demonstrates how to compare an ideal gas with a lennard jones fluid by calculating a p-V phase diagram.

## Author

Hardik Ojha

## Before you start

Make sure you installed signac. Since this work is under progress hence you'd need to clone the signac-flow repository from git into your system and install it in your virtual enviroment in editable mode.
You can follow these steps to install signac-flow:

```
git clone https://github.com/glotzerlab/signac-flow.git
pip install -e signac-flow
cd signac-flow/
git checkout feature/introduce-aggregator-classes
```

You can find the work done on the branch ``feature/introduce-aggregator-classes`` here: https://github.com/glotzerlab/signac-flow/pull/336

In [ ]:
import random

import signac
from flow import FlowProject, Aggregate, make_aggregates
import matplotlib.pyplot as plt
import numpy as np

# Enter the signac project directory
project = signac.init_project('AggregationTutorialProject', 'projects/tutorial-aggregation')

# Creating an operation

We aim to generate a plot of temperatures (in °C) v/s days of a month having 31 days. After that, we compare that plot with the average temperature of that month.

Firstly, we will create an operation which is supposed to aggregate all jobs, using the ``Aggregate`` class, and plot the data, using ``matplotlib``. 

In [ ]:
class MyProject(FlowProject):
    pass

@Aggregate(sort='day')
@MyProject.operation
def plot_average_temperature(*jobs):
    temp = [job.sp.temperature for job in jobs]
    days = [job.sp.day for job in jobs]
    average_temp = sum(temp)/len(temp)
    plt.figure(figsize=(7, 7))
    plt.plot(days, [average_temp]*len(days), 'b')
    plt.plot(days, temp, 'r')
    plt.xlabel('Day')
    plt.ylabel('Temperature in °C')
    plt.legend(['Monthly Average Temperature', 'Daily Average Temperature'])
    plt.show()

We need to use the `get_project()` *class method* to get a project handle for this special project class.

In [ ]:
project = MyProject.get_project(root='projects/tutorial-aggregation')

We need to generate random temperature data for 31 days to insert into signac jobs

In [ ]:
days = 31
avg_temp = 10 + np.random.rand() * 20
seasonal_variation = -np.cos(days / 12 * 2 * np.pi) * days
random_variation = np.random.rand(days)
temps = avg_temp + seasonal_variation + random_variation

Since we generated some data in the above cell, let's initialize the data space!


In [ ]:
for i, temp in enumerate(temps):
    sp = dict(day=i+1, temperature=temp)
    project.open_job(sp).init()

signac-flow provides a method ``make_aggregates`` which helps users to generate all the aggregates for an operation using the ``Aggregate`` object associated with it. 

So, let's create aggregates!

In [ ]:
aggregates = make_aggregates(project, plot_average_temperature)

Let's now plot the temperatures!

In [ ]:
for aggregate in aggregates:
    plot_average_temperature(*aggregate)

Uncomment and execute the following line to remove all data and start over.

In [1]:
# rm -r projects/tutorial-aggregation/workspace